# Implementation of a Feistel Cipher

This implementation of a generic Feistel cipher only serves teaching purposes. 

It does not - _by any means_ - implement a secure encryption algorithm. It is meant to serve our understanding regarding the design of a really secure algorithm.

We set our block size.

In [44]:
B_BITS = 128
B_BYTES = B_BITS // 8

ROUNDS = 16

Let's define some very simple "f" functions to facilitate our understanding.

In [45]:
def f_const_0(r, k_i):
    return b"\x00" * (B_BYTES // 2)


def f_const_1(r, k_i):
    return b"\x01" * (B_BYTES // 2)


def f_xor(r, k_i):
    return bytes([x ^ y for x, y in zip(r, k_i)])

We need a simple function to get a round key (`k_i`). In this case, we perform a simple shift operation - shifting a whole byte.

In [46]:
from collections import deque


def k_i(master_key, i, mode):
    r_i = deque(master_key)
    if mode == "encrypt":
        r_i.rotate(i)
    else:
        r_i.rotate(-i-1) # -1 here, because the first round has index "0"
    return bytes(r_i)

In [47]:
from binascii import hexlify


def round(i, l_i, r_i, f_i, k_i):
    substitution = f_i(r_i, k_i)
    r_pi = bytes([r_b ^ r_k_b for r_b, r_k_b in zip(l_i, substitution)])
    print(
        f"{i:2d} "
        f"l = {hexlify(l_i)}, r = {hexlify(r_i)}, k = {hexlify(k_i)}, f_i(r_i,k_i) = {hexlify(substitution)}, {hexlify(r_pi)}"
    )
    return (r_i, r_pi)

In [48]:
def process_block(m, f, master_k, mode):
    l = m[0 : (B_BYTES // 2)]
    r = m[(B_BYTES // 2) :]
    for i in range(0, ROUNDS):
        (l, r) = round(i, l, r, f, k_i(master_k, i, mode))

    return r + l

For the time beeing we don't care about messages which don't have a  size that is not a multiple of the block size.

In [49]:
def process_message(m_in, f, k, mode):
    assert len(m_in) % B_BYTES == 0

    m_out = b""
    for b in range(0, len(m_in) // B_BYTES):
        m_out += process_block(m_in[b * B_BYTES : (b + 1) * B_BYTES], f, k, mode)
    return m_out

Derive a master key by trimming or padding the user key. (Better: use hashing)

In [50]:
def master_k(key_candidate):
    if (len(key_candidate) <= B_BYTES//2):
        return bytes(key_candidate + "\x00"*(B_BYTES//2-len(key_candidate)),"ascii")
    else:
        return bytes(key_candidate[0:B_BYTES//2],"ascii")

In [54]:
#pwd = "A Password!"
pwd = "A"*B_BYTES
key = master_k(pwd)
m_16x8_0s = b"\x00"*B_BYTES
m_16x8 = b"This is a test.."
m_64x8 = b"This is a very large test..... not so large after all, isn't it?"
m = m_16x8
print(f"P: {hexlify(m)}, {m}")
C = process_message(m, f_xor, key, "encrypt")
print(f"C: {hexlify(C)}, {C}")
# P = process_message(C, f_xor, key, "decrypt")
# print(f"=> {hexlify(P)}, as string={P}")

P: b'54686973206973206120746573742e2e', b'This is a test..'
 0 l = b'5468697320697320', r = b'6120746573742e2e', k = b'4141414141414141', f_i(r_i,k_i) = b'2061352432356f6f', b'74095c57125c1c4f'
 1 l = b'6120746573742e2e', r = b'74095c57125c1c4f', k = b'4141414141414141', f_i(r_i,k_i) = b'35481d16531d5d0e', b'5468697320697320'
 2 l = b'74095c57125c1c4f', r = b'5468697320697320', k = b'4141414141414141', f_i(r_i,k_i) = b'1529283261283261', b'6120746573742e2e'
 3 l = b'5468697320697320', r = b'6120746573742e2e', k = b'4141414141414141', f_i(r_i,k_i) = b'2061352432356f6f', b'74095c57125c1c4f'
 4 l = b'6120746573742e2e', r = b'74095c57125c1c4f', k = b'4141414141414141', f_i(r_i,k_i) = b'35481d16531d5d0e', b'5468697320697320'
 5 l = b'74095c57125c1c4f', r = b'5468697320697320', k = b'4141414141414141', f_i(r_i,k_i) = b'1529283261283261', b'6120746573742e2e'
 6 l = b'5468697320697320', r = b'6120746573742e2e', k = b'4141414141414141', f_i(r_i,k_i) = b'2061352432356f6f', b'74095c57125c1c4f'
 7

Print some frequency information to see the inadequacy of our cipher/function f/subkey generation algo.

In [52]:
from array import array
freq = array('L',(0 for i in range(0,256)))
for i in range(0,len(C)):
    freq[C[i]] += 1

x_labels = []
x_data = []
for i in range(0,256):
    if(freq[i] > 0):
        x_labels.append(i)
        x_data.append(freq[i])
        print(f"{i:2x}", "=>", freq[i])

 4 => 2
1b => 2
1f => 2
2a => 1
54 => 1
5e => 1
68 => 1
6b => 1
6c => 1
74 => 1
77 => 1
7a => 1
7e => 1
